In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp



In [2]:
#parameters

frames = 45
echo_cato = ["ASD","DCM","HP","MI","NORM"]
number = 1000 # one sample


dir_ = "...."


In [3]:
#function

def generate_npy(txt_dir, data_dir, frames, clips_no=5):
    """
    channel last as keras
    
    """
    
    # read txt file
    txt = pd.read_csv(txt_dir,header = None)
    txt = txt.rename(index=str, columns={0: "photo_name"})
    txt["dir_name"] = ""
    txt["rank"] = ""
    
    # k = 0
    for i, name in enumerate(txt["photo_name"]):    
        name_list = name[:-4].split("_")
        txt["rank"][i] = float(name_list[-1])
        length = 4 + len(name_list[-1]) + 1
        txt["dir_name"][i] = name[:-length]
        #k = k + 1
        #if k>50:
        #    break;
    
    txt = pd.DataFrame(txt)
    
    # read data file
    
    data = pd.read_csv(data_dir,header = None)
    feat_cols = [ 'pixel_'+str(i) for i in range(data.shape[1]) ]
    data = pd.DataFrame(data.values,columns=feat_cols)
    
    data = pd.DataFrame(data)
    
    
    # merge data and txt togather
    
    data2 = data.copy()
    data2["photo_name"] = ""
    data2["dir_name"] = ""
    data2["rank"] = "0"
    
    for i, value in enumerate(txt.iloc[:,0]):
        data2.loc[i,"photo_name"] = value
    for i, value in enumerate(txt.iloc[:,1]):
        data2.loc[i,"dir_name"] = value
    for i, value in enumerate(txt.iloc[:,2]):
        data2.loc[i,"rank"] = value  
    
    # sort data2 into the right order
    
    data2 = data2.sort_values(by=['dir_name','rank'])
    data2 = data2.reset_index(drop=True)
    
    # transfer 2048 dimension data to 512*15 data
    
    sample_name = data2.dir_name.unique()
    length = len(sample_name)
    n_array = np.zeros(shape=(length*clips_no, frames, data.shape[1]))
    
    
    count = 0
    for name in data2.dir_name.unique():
        (x,y) = data2[data2["dir_name"]== name].shape
        npy = data2[data2["dir_name"]== name].iloc[:,:data.shape[1]].values
        if x<frames:
            kk = 0
            for cn in range(clips_no):
                for i in range(frames):
                    n_array[count,i,::] = npy[i%x]
                count = count + 1
        elif (x>=frames)&(x<(frames+clips_no-1)):
            kk = 1
            for cn in range(clips_no):
                t = cn%(x-frames+1)
                for i in range(frames):
                    tt = t+i
                    n_array[count,i,::] = npy[tt]
                count = count + 1
        elif x>=(frames+clips_no-1):
            kk = 2
            for cn in range(clips_no):
                clips_2 = float(clips_no - 1)
                t = int((x-frames)/clips_2*cn)
                for i in range(frames):
                    tt = t+i
                    n_array[count,i,::] = npy[tt]
                count = count + 1
    print(count, length)
    return n_array
    

# 1. generate training, validation and testing data

In [ ]:
record = {}
for index, name in enumerate(echo_cato):
    
    print(name)
    txt_dir = dir_ + "train_" + name + "_name.txt"
    data_dir = dir_ + "train_" + name + "_data.txt"
    frames = frames
    # clips_no=5
    array = generate_npy(txt_dir, data_dir, frames, clips_no=5)
    
    if array.shape[0]<400:
        array = np.concatenate((array, array), axis=0)
    
    record[name] = array.shape
    y_array = np.ones(shape=(array.shape[0],1))
    y_array = y_array*(index)
    
    if index == 0:
        X_train = array.copy()
        y_train = y_array.copy()
    else:
        X_train = np.concatenate((X_train, array), axis=0)
        y_train = np.concatenate((y_train, y_array), axis=0)


record = {}
for index, name in enumerate(echo_cato):
    
    print(name)
    txt_dir = dir_ + "validation" + name + "_name.txt"
    data_dir = dir_ + "validation" + name + "_data.txt"
    frames = frames
    # clips_no=5
    array = generate_npy(txt_dir, data_dir, frames, clips_no=5)
    if array.shape[0]<400:
        array = np.concatenate((array, array), axis=0)
    
    record[name] = array.shape
    y_array = np.ones(shape=(array.shape[0], 1))
    y_array = y_array*(index)
    
    if index == 0:
        X_val = array.copy()
        y_val = y_array.copy()
    else:
        X_val = np.concatenate((X_test, array), axis=0)
        y_val = np.concatenate((y_test, y_array), axis=0)


record = {}
for index, name in enumerate(echo_cato):
    
    print(name)
    txt_dir = dir_ + "test" + name + "_name.txt"
    data_dir = dir_ + "test" + name + "_data.txt"
    frames = frames
    # clips_no=5
    array = generate_npy(txt_dir, data_dir, frames, clips_no=5)
    if array.shape[0]<400:
        array = np.concatenate((array, array), axis=0)
    
    record[name] = array.shape
    y_array = np.ones(shape=(array.shape[0], 1))
    y_array = y_array*(index)
    
    if index == 0:
        X_test = array.copy()
        y_test = y_array.copy()
    else:
        X_test = np.concatenate((X_test, array), axis=0)
        y_test = np.concatenate((y_test, y_array), axis=0)

# 2. plot one sample

In [ ]:
one_sample = sns.clustermap(X_train[number,::,::].T, 
                   col_cluster=False, 
                   # row_cluster=False, 
                   z_score=0, 
                   robust = True, 
                   cmap="coolwarm",
                   xticklabels=False, yticklabels=False,
                   # square=True,
                   figsize=(10,10)
                  )
one_sample.savefig('....pdf', format="pdf", dpi=400)

# 3. build a network for diagnoisis

In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
import os

from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import (
    ReduceLROnPlateau,
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint)

In [ ]:
# parameters:

BZ = 250

LR = 0.001

RG = 50

CLASS = 5

X_train = X_train.astype('float32')
y_train = y_train.reshape((y_train.shape[0],))
y_train = y_train.astype('float32')
X_val = X_val.astype('float32')
y_val = y_val.reshape((y_test.shape[0],))
y_val = y_val.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, CLASS)
y_val = keras.utils.to_categorical(y_val, CLASS)


In [ ]:
model = Sequential()
model.add(Conv1D(2048, 2, activation='relu',padding = "same", input_shape=(X_train.shape[1],2048)))
model.add(Conv1D(512, 2, activation='relu', dilation_rate = 2, padding = "same"))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5,seed = 42))
model.add(Dense(CLASS, activation='softmax'))

model.compile(optimizer=SGD(lr=LR, momentum=0.9), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)
# Checkpoint
checkpointer = ModelCheckpoint(
    filepath="model_{}_checkpoint_{}_{}.h5".format(type_name, "first", "title"),
    verbose=1,
    save_best_only=True)

# csvlogger
csv_logger = CSVLogger(
    'csv_logger_{}_{}_{}.csv'.format(type_name, "first", "title"))

In [ ]:
model.fit(X_train, y_train, 
          epochs= 50, 
          validation_data = (X_val, y_val), callbacks=[csv_logger, checkpointer])

# 4. plot ROC curves (code from scikit-learn)

In [ ]:
from keras.models import load_model

model_name = "model_45X2048.h5"
model = load_model(model_name)

y_score = model.predict(X_test)
n_classes = y_score.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area
lw=2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area
lw=2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.5f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.5f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.5f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()